#### Importing necessary libraries for data cleaning and extraction

In [1]:
import numpy as np
import pandas as pd
import re

In [3]:
data = pd.read_csv("C:\\Users\\kumar\\Downloads\\mobilephonedataset\\Flipkart_Mobiles_Dataset.csv")
data.head()

,Brand,Model,Color,Memory,Storage,Rating,Selling Price,Original Price,Total_sales
0,OPPO,A53,Moonlight Black,4 GB,64 GB,4.5,11990,15990,1563
1,OPPO,A53,Mint Cream,4 GB,64 GB,4.5,11990,15990,4909
2,OPPO,A53,Moonlight Black,6 GB,128 GB,4.3,13990,17990,701
3,OPPO,A53,Mint Cream,6 GB,128 GB,4.3,13990,17990,5241
4,OPPO,A53,Electric Black,4 GB,64 GB,4.5,11990,15990,8837


In [4]:
# Cleaning the headers by changing case and removing spaces
def change_col():
    new_col = []
    for i in data.columns:
        j= i.lower()
        k= j.replace(" ","_")
        new_col.append(k)
    return new_col
data.columns  = change_col()
data.columns    

Index(['brand', 'model', 'color', 'memory', 'storage', 'rating',
       'selling_price', 'original_price', 'total_sales'],
      dtype='object')

In [5]:
data.duplicated(["brand","model","color","memory","storage"]).sum()

201

In [6]:
# Dropping duplicates
data.drop_duplicates(subset=["brand","model","color","memory","storage"],inplace=True)

In [7]:
data.shape  ,   data.size

((2913, 9), 26217)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2913 entries, 0 to 3113
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   brand           2913 non-null   object 
 1   model           2913 non-null   object 
 2   color           2913 non-null   object 
 3   memory          2882 non-null   object 
 4   storage         2874 non-null   object 
 5   rating          2787 non-null   float64
 6   selling_price   2913 non-null   int64  
 7   original_price  2913 non-null   int64  
 8   total_sales     2913 non-null   int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 227.6+ KB


In [9]:
data.isna().sum()

brand               0
model               0
color               0
memory             31
storage            39
rating            126
selling_price       0
original_price      0
total_sales         0
dtype: int64

In [10]:
# Dropping NULL values
data.dropna(how="any",subset=["memory","storage","rating"],inplace=True)

In [11]:
data.rename(columns={"memory":"RAM","storage":"ROM"},inplace=True)

In [12]:
data.ROM.unique() ,  data.RAM.unique()

(array(['64 GB', '128 GB', '32 GB', '256 GB', '16 GB', '8 GB', '4 GB',
        '512 GB', '16 MB', '128 MB', '2 MB', '4 MB',
        'Expandable Upto 32 GB', '8 MB', 'Expandable Upto 16 GB', '10 MB',
        '256 MB', '140 MB', '64 MB', '153 MB', '512 MB', '1 TB', '100 MB',
        '129 GB', '130 GB'], dtype=object),
 array(['4 GB', '6 GB', '3 GB', '8 GB', '2 GB', '12 GB', '1 GB', '512 MB',
        '1.5 GB', '768 MB', '8 MB', '64 MB', '4 MB', '32 MB', '16 MB',
        '4GB', '153 MB', '128 MB', '2 MB', '10 MB', '46 MB', '16 GB',
        '100 MB', '30 MB'], dtype=object))

In [13]:
data.head()

,brand,model,color,RAM,ROM,rating,selling_price,original_price,total_sales
0,OPPO,A53,Moonlight Black,4 GB,64 GB,4.5,11990,15990,1563
1,OPPO,A53,Mint Cream,4 GB,64 GB,4.5,11990,15990,4909
2,OPPO,A53,Moonlight Black,6 GB,128 GB,4.3,13990,17990,701
3,OPPO,A53,Mint Cream,6 GB,128 GB,4.3,13990,17990,5241
4,OPPO,A53,Electric Black,4 GB,64 GB,4.5,11990,15990,8837


In [14]:
# We only need data that has the storage in GB which refers that these are smart phones ,
# so we create a new column for further deleting the models with RAM & ROM with other than GB

col_RAM_del_list = []
col_ROM_del_list = []
for i in data.RAM:
    if 'GB' in i:
        col_RAM_del_list.append("OK")
    else:
        col_RAM_del_list.append(np.nan)
        
for j in data.ROM:
    if 'GB' in j:
        col_ROM_del_list.append("OK")
    else:
        col_ROM_del_list.append(np.nan)
        
data["del_RAM"] = col_RAM_del_list
data["del_ROM"] = col_ROM_del_list

In [15]:
data.isna().sum()

brand               0
model               0
color               0
RAM                 0
ROM                 0
rating              0
selling_price       0
original_price      0
total_sales         0
del_RAM           117
del_ROM            68
dtype: int64

In [16]:
# dropping the rows that has null values , i.e which have RAM & ROM other that GB 
data.dropna(how="any",subset=["del_RAM","del_ROM"],inplace=True)

In [17]:
# dropping the columns "del_RAM" and "del_ROM"
data.drop(columns=["del_RAM","del_ROM"],inplace=True)

In [18]:
# Removing the text part in the RAM & ROM columns 
data["ROM"] = data["ROM"].str.extract(r'(\d+)')
data["RAM"] = data["RAM"].str.extract(r'(\d+)')

In [19]:
# creating a new column by concating "RAM" & "ROM" as "storage" for obtaining unique values of storage for further process
data["storage"] = data["RAM"]+"_"+data["ROM"]
data.head(2)

,brand,model,color,RAM,ROM,rating,selling_price,original_price,total_sales,storage
0,OPPO,A53,Moonlight Black,4,64,4.5,11990,15990,1563,4_64
1,OPPO,A53,Mint Cream,4,64,4.5,11990,15990,4909,4_64


In [20]:
data["storage"].unique()

array(['4_64', '6_128', '3_32', '8_128', '2_32', '4_128', '12_256',
       '6_256', '8_256', '6_64', '3_64', '2_16', '4_32', '3_16', '1_16',
       '1_8', '1_4', '12_128', '2_8', '12_512', '1_32', '2_64', '3_128',
       '2_128', '4_256', '2_256', '6_512', '3_256', '4_512', '1_64',
       '1_128', '8_512', '16_512', '6_129', '6_130', '8_129', '8_130'],
      dtype=object)

In [21]:
data.head()

,brand,model,color,RAM,ROM,rating,selling_price,original_price,total_sales,storage
0,OPPO,A53,Moonlight Black,4,64,4.5,11990,15990,1563,4_64
1,OPPO,A53,Mint Cream,4,64,4.5,11990,15990,4909,4_64
2,OPPO,A53,Moonlight Black,6,128,4.3,13990,17990,701,6_128
3,OPPO,A53,Mint Cream,6,128,4.3,13990,17990,5241,6_128
4,OPPO,A53,Electric Black,4,64,4.5,11990,15990,8837,4_64


In [22]:
# Adding new column with incremented values as "Model number"
data.insert(0,"model_type_id",range(1,len(data)+1),allow_duplicates=False)

In [23]:
# Creating seperate table as model type with model number , brand and model columns as attributes
model_type_df  =  pd.DataFrame(data[["model_type_id","brand","model"]])
model_type_df.head()

,model_type_id,brand,model
0,1,OPPO,A53
1,2,OPPO,A53
2,3,OPPO,A53
3,4,OPPO,A53
4,5,OPPO,A53


In [25]:
# Creating a new column major_coor according to the correspoding color code in major-color list
c_list = list(data['color'].squeeze())
color_col_list= []
for i in c_list:
    color_col_list.append(i.lower())

major_colour = ["blue","black","brown","red","gold","purple","green","white","grey","silver","pink","yellow","orange","violet"] 
new_color_col_list = []
for i in color_col_list:
    for j in major_colour:
        if j in i:
            new_color_col_list.append(j)
            break 
    else:
        new_color_col_list.append("others")


In [26]:
data.head(3)

,model_type_id,brand,model,color,RAM,ROM,rating,selling_price,original_price,total_sales,storage
0,1,OPPO,A53,Moonlight Black,4,64,4.5,11990,15990,1563,4_64
1,2,OPPO,A53,Mint Cream,4,64,4.5,11990,15990,4909,4_64
2,3,OPPO,A53,Moonlight Black,6,128,4.3,13990,17990,701,6_128


In [27]:
# Inserting new column main_color to the data set
data.insert(4,"main_colour",new_color_col_list)

In [28]:
data.head(3)

,model_type_id,brand,model,color,main_colour,RAM,ROM,rating,selling_price,original_price,total_sales,storage
0,1,OPPO,A53,Moonlight Black,black,4,64,4.5,11990,15990,1563,4_64
1,2,OPPO,A53,Mint Cream,others,4,64,4.5,11990,15990,4909,4_64
2,3,OPPO,A53,Moonlight Black,black,6,128,4.3,13990,17990,701,6_128


In [29]:
# Creating seperate table as model color with color_id , color and main_color columns as attributes
color_df  =  pd.DataFrame(data[["color","main_colour"]])

color_df.duplicated('color').sum()

1985

In [30]:
# Remove duplicates from model_color_df
color_df.drop_duplicates(subset=['color'],inplace=True)

In [31]:
color_df.shape

(617, 2)

In [32]:
# Inserting new column color_id to the data set 
color_df.insert(0,"color_id",range(1,len(color_df)+1))

In [33]:
color_df.head(3)

,color_id,color,main_colour
0,1,Moonlight Black,black
1,2,Mint Cream,others
4,3,Electric Black,black


In [34]:
# Merging data with model_color_df and then changing the original df 'data' to the obtained merged data
data_with_colortbl_mrg = data.merge(color_df, on="color",sort=False)
data = data_with_colortbl_mrg.loc[::,['model_type_id', 'brand', 'model', 'color', 'main_colour_x', 'RAM', 'ROM',
       'rating', 'selling_price', 'original_price', 'total_sales', 'storage',
       'color_id']]

In [35]:
data.head(3)

,model_type_id,brand,model,color,main_colour_x,RAM,ROM,rating,selling_price,original_price,total_sales,storage,color_id
0,1,OPPO,A53,Moonlight Black,black,4,64,4.5,11990,15990,1563,4_64,1
1,3,OPPO,A53,Moonlight Black,black,6,128,4.3,13990,17990,701,6_128,1
2,18,OPPO,A33,Moonlight Black,black,3,32,4.3,10490,12990,8683,3_32,1


In [36]:
storage_df =pd.DataFrame(data[["storage","RAM","ROM"]])
storage_df.duplicated('storage').sum()

2565

In [37]:
# Removing duplicates from storage dataframe with certain condition 
storage_df.drop_duplicates(subset=['storage'],inplace=True)

In [38]:
# Inserting new column to the storage_df
storage_df.insert(0,"storage_id",range(1,len(storage_df)+1))

In [39]:
storage_df.head(3)

,storage_id,storage,RAM,ROM
0,1,4_64,4,64
1,2,6_128,6,128
2,3,3_32,3,32


In [40]:
# Merging data with model_storage_df and then changing the original df 'data' to the obtained merged data
data_with_storagetbl_mrg = data.merge(storage_df, on="storage",sort=False)
data = data_with_storagetbl_mrg.loc[::,['model_type_id', 'brand', 'model', 'color', 'main_colour_x', 'RAM_x',
       'ROM_x', 'rating', 'selling_price', 'original_price', 'total_sales',
       'storage', 'color_id', 'storage_id']]

In [41]:
data.head(3)

,model_type_id,brand,model,color,main_colour_x,RAM_x,ROM_x,rating,selling_price,original_price,total_sales,storage,color_id,storage_id
0,1,OPPO,A53,Moonlight Black,black,4,64,4.5,11990,15990,1563,4_64,1,1
1,2,OPPO,A53,Mint Cream,others,4,64,4.5,11990,15990,4909,4_64,2,1
2,5,OPPO,A53,Electric Black,black,4,64,4.5,11990,15990,8837,4_64,3,1


In [42]:
# Creating new tabel(dataframe) rating_df with certain values directly inserted
rating_df = pd.DataFrame({
                         "rating_id" : [1,2,3,4],
                         "rating_desc" : ['poor', 'average', 'above-average' ,'excellent']
                        })
rating_df

,rating_id,rating_desc
0,1,poor
1,2,average
2,3,above-average
3,4,excellent


In [43]:
# Creating rating_id column and values within according to the rating of model given 
conditions = [(data['rating'] >=1) & (data['rating'] < 2),
              (data['rating'] >=2) & (data['rating'] < 3),
              (data['rating'] >=3) & (data['rating'] < 4),
              (data['rating'] >=4) & (data['rating'] <= 5)]
values = [1,2,3,4]
data['rating_id'] = np.select(conditions,values)
data.head(2)

,model_type_id,brand,model,color,main_colour_x,RAM_x,ROM_x,rating,selling_price,original_price,total_sales,storage,color_id,storage_id,rating_id
0,1,OPPO,A53,Moonlight Black,black,4,64,4.5,11990,15990,1563,4_64,1,1,4
1,2,OPPO,A53,Mint Cream,others,4,64,4.5,11990,15990,4909,4_64,2,1,4


In [44]:
# Filtering data and sorting values according to model_type_id and inserting new column  sl_no
model_color_df = pd.DataFrame(data[["model_type_id","color_id"]])
model_color_df.sort_values(by="model_type_id",inplace=True)
model_color_df.insert(0,"sl_no",range(1,len(model_color_df)+1))
model_color_df.head()

,sl_no,model_type_id,color_id
0,1,1,1
1,2,2,2
527,3,3,1
528,4,4,2
2,5,5,3


In [45]:
# Inserting new column sl_no and creating dataframe with given columns as data by filtering out
model_storage_df = pd.DataFrame(data[["model_type_id","storage_id"]])
model_storage_df.sort_values(by="model_type_id",inplace=True)
model_storage_df.insert(0,"sl_no",range(1,len(model_storage_df)+1))
model_storage_df.head(3)

,sl_no,model_type_id,storage_id
0,1,1,1
1,2,2,1
527,3,3,2


In [46]:
# Creating new dataframe model_rating_df and new colum,n sl_no and sorting the rows on model_type_id
model_rating_df = pd.DataFrame(data[["model_type_id","rating","rating_id"]])
model_rating_df.sort_values(by="model_type_id",inplace=True)
model_rating_df.insert(0,"sl_no",range(1,len(model_rating_df)+1))
model_rating_df.head(3)

,sl_no,model_type_id,rating,rating_id
0,1,1,4.5,4
1,2,2,4.5,4
527,3,3,4.3,4


In [47]:
# Creating new dataframe mobile_sales_df and sorting the rows on model_type_df
mobile_sales_df = pd.DataFrame(data[["model_type_id","total_sales","original_price","selling_price"]])
mobile_sales_df.sort_values(by="model_type_id",inplace=True)
mobile_sales_df.head(3)

,model_type_id,total_sales,original_price,selling_price
0,1,1563,15990,11990
1,2,4909,15990,11990
527,3,701,17990,13990


In [48]:
data.head(3)

,model_type_id,brand,model,color,main_colour_x,RAM_x,ROM_x,rating,selling_price,original_price,total_sales,storage,color_id,storage_id,rating_id
0,1,OPPO,A53,Moonlight Black,black,4,64,4.5,11990,15990,1563,4_64,1,1,4
1,2,OPPO,A53,Mint Cream,others,4,64,4.5,11990,15990,4909,4_64,2,1,4
2,5,OPPO,A53,Electric Black,black,4,64,4.5,11990,15990,8837,4_64,3,1,4


In [49]:
model_type_df = data.loc[::,['brand','model','color_id','storage_id']]

In [50]:
model_type_df.head(3)

,brand,model,color_id,storage_id
0,OPPO,A53,1,1
1,OPPO,A53,2,1
2,OPPO,A53,3,1


In [51]:
model_type_df.count()

brand         2602
model         2602
color_id      2602
storage_id    2602
dtype: int64

In [52]:
model_rating_df.count()

sl_no            2602
model_type_id    2602
rating           2602
rating_id        2602
dtype: int64

In [54]:
model_rating_df = model_rating_df.iloc[::,1::]

In [55]:
model_rating_df.head(2)

,model_type_id,rating,rating_id
0,1,4.5,4
1,2,4.5,4


In [67]:
rating_df = rating_df.iloc[::,1::]
rating_df.head(2)

,rating_desc
0,poor
1,average


In [57]:
rating_df.count()

rating_id      4
rating_desc    4
dtype: int64

In [65]:
color_df = color_df.iloc[::,1::]
color_df.head(3)

,color,main_colour
0,Moonlight Black,black
1,Mint Cream,others
4,Electric Black,black


In [59]:
color_df.duplicated().sum() , color_df.count()

(0,
 color_id       617
 color          617
 main_colour    617
 dtype: int64)

In [60]:
mobile_sales_df.head(2)

,model_type_id,total_sales,original_price,selling_price
0,1,1563,15990,11990
1,2,4909,15990,11990


In [61]:
mobile_sales_df.count() , mobile_sales_df.duplicated().sum()

(model_type_id     2602
 total_sales       2602
 original_price    2602
 selling_price     2602
 dtype: int64,
 0)

In [62]:
storage_df.head(2)

,storage_id,storage,RAM,ROM
0,1,4_64,4,64
1,2,6_128,6,128


In [63]:
storage_df = storage_df.loc[::,['storage_id','RAM','ROM']]

In [66]:
storage_df = storage_df.iloc[::,1::]
storage_df.head(2)

,RAM,ROM
0,4,64
1,6,128


#### Converting the dataframe to CSV files without indexes and headers for further database building process

In [69]:
mobile_sales_df.to_csv("C:\\Users\\kumar\\Downloads\\mobile_dfcsv_values\\mobile_sales_csv.csv",index=False,header=False)
model_rating_df.to_csv("C:\\Users\\kumar\\Downloads\\mobile_dfcsv_values\\model_rating_csv.csv",index=False,header=False)
rating_df.to_csv("C:\\Users\\kumar\\Downloads\\mobile_dfcsv_values\\rating_csv.csv",index=False,header=False)
storage_df.to_csv("C:\\Users\\kumar\\Downloads\\mobile_dfcsv_values\\storage_csv.csv",index=False,header=False)
color_df.to_csv("C:\\Users\\kumar\\Downloads\\mobile_dfcsv_values\\color_csv.csv",index=False,header=False)
model_type_df.to_csv("C:\\Users\\kumar\\Downloads\\mobile_dfcsv_values\\model_type_csv.csv",index=False,header=False)